In [5]:
# Notebook for more data investigation
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
cmap = plt.cm.get_cmap('Dark2',7)
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [6]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

# latestdir = rootdir+ '/SSI_data_2021-04-29'
# latestDate = pd.to_datetime(latestdir[-10:])

# print(latestdir)
# print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-05-21
2021-05-21 00:00:00


In [7]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldfAG = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# # Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
dfAG = fulldfAG.iloc[:-2,:].copy()

In [8]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
dfAG["PosPct"] = pd.to_numeric(dfAG["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG["Date"] = dfAG["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    dfAG[name] = pd.to_numeric(dfAG[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [9]:
# Load the data

dfAG2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

dfAG2["Dato"] = dfAG2["Dato"].astype('datetime64[D]')
dfAG2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [10]:
# Cut off early data in pcr dataframe
dfPCR = dfPCR[dfPCR.Date >= dfAG.iloc[0].Date]

In [11]:
# Also cutoff everything before first of January
firstDate = np.datetime64('2021-02-01')
dfPCR = dfPCR[dfPCR.Date >= firstDate]
dfAG = dfAG[dfAG.Date >= firstDate]

In [12]:
# Cut off last date (since it is not fully done)
dfPCR = dfPCR.iloc[:-1]
dfAG = dfAG.iloc[:-1]

In [13]:
# Get dates
curDates = dfPCR.Date.values.astype('datetime64[D]')
confDates = dfAG2.Dato.values.astype('datetime64[D]')

In [14]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
474,2021-05-15,941,135868,0.7,3983,139851,29898129
475,2021-05-16,922,163930,0.6,4366,168296,30066425
476,2021-05-17,1147,196526,0.6,6602,203128,30269553
477,2021-05-18,1160,173301,0.7,5706,179007,30448560
478,2021-05-19,1130,170322,0.7,5199,175521,30624081


In [15]:
dfAG.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
402,2021-05-15,309,262938,0.1,789,263727,14403715
403,2021-05-16,379,304545,0.1,909,305454,14709169
404,2021-05-17,454,493180,0.1,1502,494682,15203851
405,2021-05-18,383,391221,0.1,1153,392374,15596225
406,2021-05-19,352,388997,0.1,1160,390157,15986382


In [16]:
dfAG2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
101,2021-05-13,295174,399,364,91.228070,35,8.771930,210,147,189,17237
102,2021-05-14,323429,406,371,91.379310,35,8.620690,219,148,244,18794
103,2021-05-15,263817,346,305,88.150289,41,11.849711,187,115,232,18264
104,2021-05-16,305579,421,383,90.973872,38,9.026128,262,115,276,19956
105,2021-05-17,494948,499,473,94.789579,26,5.210421,283,178,413,27276


In [13]:
# 754 * ((30000/139851)**0.7)

# agpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']).iloc[-14:].mean()
AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']).iloc[-14:].mean()

confPCRpos = dfAG2.AGpos_PCRpos + dfAG2.AGnegPCRpos
AG_TPR = np.divide(dfAG2.AGpos_PCRpos,confPCRpos).iloc[-14:].mean()
(AGpospct2 / AG_TPR) * 5800000 / 3
# AGpospct2 * 100
# AG_TPR
# (AGpospct2 / AG_TPR)
# (AGpospct2 / AG_TPR) * 5800000

2842.410460629746

In [17]:
# Overblik over antal tests og antal positive

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1)

PCRpos = dfPCR.NewPositive
AGpos = dfAG.NewPositive
PCRtest = dfPCR.Tested
AGtest = dfAG.Tested
ax1.plot(curDates,PCRtest,label='PCR')
ax1.plot(curDates,AGtest,label='AG')
ax2.plot(curDates,PCRpos,label='PCR')
ax2.plot(curDates,AGpos,label='AG')
# ax2.plot(curDates,np.multiply(PCRpos,PCRtest),label='PCR')
# ax2.plot(curDates,np.multiply(AGpos,AGtest),label='AG')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))


ax1.legend()
ax2.legend()
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# Overblik over hvor mange AG-tests der er positive
AGposConfRate = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede'])
AGposConfPerConf = np.divide(AGposConfRate,0.01*dfAG2['Andel_AGpos_PCRkonf_pct'])

# testArray = np.divide(np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']),0.01*dfAG2['Andel_AGpos_PCRkonf_pct'])*100

agPosPCT = np.divide(dfAG.NewPositive,dfAG.Tested)
fig,ax1 = plt.subplots()

ax1.plot(confDates,100 * AGposConfPerConf,'*-',label='PCR-konfirmeret andel af sande positive AG tests')
# plt.plot(curDates,dfAG.PosPct)
ax1.plot(curDates,100 * agPosPCT,'*-',label='AG test, positiv procent')
ax1.set_ylim([0,0.4])
ax1.set_xlim(left = np.datetime64('2021-03-01'))
ax1.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Method 1

In [19]:
# curDates.iloc[-10:]

# PCRpos = dfPCR.NewPositive
# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR.NewPositive.values
numPCR = dfPCR.Tested.values
AGpos = dfAG.NewPositive.values
numAG = dfAG.Tested.values
AGposPCRpos = dfAG2.AGpos_PCRpos.values

PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

In [20]:
# Calculate the true positive rate
AGtruePos = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
avgTruePos = AGtruePos.iloc[-14:].mean()
avgConf = 0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].iloc[-14:].mean()
# avgTruePos


# AGproj = np.divide(avgTruePos*dfAG2['AG_pos'],0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].values)
AGproj = np.divide(avgTruePos*AGpos,avgConf)

confDatesProj = np.concatenate([confDates[:],curDates[-2:]])
confDatesProj = np.append(confDates[:],curDates[-2:]).astype('datetime64[D]')
AGposPCRposProj = np.append(AGposPCRpos,AGproj[-2:])

In [21]:

# fig,ax1 = plt.subplots(1,1,sharex=True)
# ax1.plot(confDates,AGtruePos)

confDatesProj[0]
curDates[-1]

numpy.datetime64('2021-05-19')

In [22]:
firstDate = np.datetime64('2021-03-01')
lastDate = curDates[-1]

fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,AGpos,'.-',color='xkcd:green',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGpos,meanWidth),color='xkcd:green',label='AG positive, total')

ax1.plot(confDatesProj,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
ax1.plot(confDatesProj[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
ax1.plot(rnTime(confDatesProj,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')


ax1.legend(loc='upper left')
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax1.set_ylim(bottom=0,top=500)
ax1.set_ylabel('Antal')

# ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
ax1.set_xlim([firstDate,lastDate])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Screening_AGconfirmed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig,ax1 = plt.subplots(1,1,sharex=True)

PCRpos_nonAG = PCRpos - AGposPCRposProj

ax1.plot(curDates,PCRpos,'.-',color='xkcd:brown',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:brown',label='PCR positive, alle')

ax1.plot(curDates,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
ax1.plot(curDates[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')


ax1.plot(curDates,PCRpos_nonAG,'.-',color='xkcd:dark blue',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color='xkcd:dark blue',label='PCR positive, uden AG bekræftning')

ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylim(bottom=0,top=1500)
ax1.set_ylabel('Antal')

ax1.set_xlim([firstDate,lastDate])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Screening_PCRnonAG')

# ax1.plot(curDates,(1/0.9)*avgTruePos*AGpos,':',color='xkcd:dark green',label='AG positive, projekteret')
# ax1.plot(confDates,0.9*dfAG2['AG_pos'],'.-',color='xkcd:dark green',linewidth=0.5,markersize=2,label='AG positive, to be confirmed')
# ax1.plot(confDates,AGproj,'.-',color='xkcd:dark red',linewidth=0.5,markersize=2,label='AG positive, to be asdf')
# ax1.plot(curDates[-3:],AGproj[-3:],'*-',color='xkcd:dark green',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(curDates,meanWidth)[-3:],rnMean(AGproj,meanWidth)[-3:],':',color='xkcd:dark green',label='AG positive, projektion')
# ax1.plot(confDates,AGposPCRpos,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2,label='AG positive, PCR bekræftede')
# ax1.plot(rnTime(confDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:dark green')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig,ax1 = plt.subplots(1,1,sharex=True)

PCRpos_nonAG = PCRpos - AGposPCRposProj

# ax1.plot(curDates,PCRpos,'.-',color='xkcd:brown',linewidth=0.5,markersize=2)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:brown',label='PCR positive, alle')

# ax1.plot(curDates,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
# ax1.plot(curDates[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')


ax1.plot(curDates,PCRpos_nonAG,'.-',color='xkcd:dark blue',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color='xkcd:dark blue',label='PCR positive, uden AG bekræftning')

PCRnoscreen = PCRpos_nonAG * ( (30000/dfPCR.Tested)**0.7)
PCRscreen = PCRpos_nonAG - PCRnoscreen

ax1.plot(curDates,PCRnoscreen,'.-',color='m',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRnoscreen,meanWidth),color='m',label='PCR positive, uden screening')
ax1.plot(curDates,PCRscreen,'.-',color='r',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRscreen,meanWidth),color='r',label='PCR positive, ren screening')

ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylim(bottom=0,top=1500)
ax1.set_ylabel('Antal')

ax1.set_xlim([firstDate,lastDate])
plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/Screening_PCRnonAG')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig,ax1 = plt.subplots(1,1,sharex=True)
PCRpos_nonAG = PCRpos - AGposPCRposProj


cmap = plt.cm.get_cmap('tab20',7)

# ax1.plot(curDates,PCRpos,'.-',color='xkcd:brown',linewidth=0.5,markersize=2)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:brown',label='PCR positive, alle')

# ax1.plot(curDates,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
# ax1.plot(curDates[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')


# ax1.fill(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color='xkcd:dark blue',label='PCR positive, uden AG bekræftning')

PCRnoscreen = PCRpos_nonAG * ( (30000/dfPCR.Tested)**0.7)
PCRscreen = PCRpos_nonAG - PCRnoscreen

ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(0),label='PCR positive, ved 30.000 tests')
# ax1.plot(curDates,PCRscreen,'.-',color='r',linewidth=0.5,markersize=2)
ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(1),label='PCR positive, kun screening')


# ax1.fill_between(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG+AGposPCRposProj,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color='g',label='PCR positive + AG screening')
ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRpos,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color=cmap(2),label='PCR positive, efter AG positive')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth))
# ax1.plot(curDates,PCRpos)

# ax1.plot(curDates,PCRpos_nonAG,'.-',color='k',linewidth=0.5,markersize=2)
# ax1.plot(curDates,PCRnoscreen,'.-',color='k',linewidth=0.5,markersize=2)
# ax1.plot(curDates,PCRpos,'.-',color='k',linewidth=0.5,markersize=2)

ax1.legend(loc='upper left')
ax1.grid(color='k')
ax1.set_ylim(bottom=0,top=1100)
# ax1.set_ylim(bottom=0)
ax1.set_ylabel('Antal')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

curLastDate = rnTime2(curDates,meanWidth)[-1]
ax1.set_xlim([firstDate,curLastDate])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Screening_PCRstacked')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
# Estimat af faktiske ny-smittede per dag
fig,ax1 = plt.subplots()

dkbefolk = 5800000
# AGpospct = np.divide(dfAG.NewPositive,dfAG.Tested)
# ax1.plot(curDates,AGpospct*dkbefolk)

AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede'])

PCRpospct = np.divide(dfPCR.NewPositive,dfPCR.Tested)
# AGtruePos
# AGtruePos = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
confPCRpos = dfAG2.AGpos_PCRpos + dfAG2.AGnegPCRpos
AG_TPR = np.divide(dfAG2.AGpos_PCRpos,confPCRpos)

disPeriod = 3
disPeriodPCR = 6

numInfected = (AGpospct2/AG_TPR)*dkbefolk
numInfectedPCR = PCRpospct*dkbefolk

SSI_incidens = np.divide( np.multiply(dfPCR.NewPositive,100000) , dkbefolk )
# SSI_incidens = dkbefolk * dfPCR.NewPositive / (100000)
# SSI_incidens = dkbefolk * dfPCR.NewPositive / (100000)

ax1.plot(curDates,numInfectedPCR/disPeriodPCR,'.-',color='b',linewidth=0.5,markersize=2)
ax1.plot(rnTime2(curDates,meanWidth),rnMean(numInfectedPCR/disPeriodPCR,meanWidth),'b')
ax1.plot(confDates,numInfected/disPeriod,'.-',color='r',linewidth=0.5,markersize=2)
ax1.plot(rnTime2(confDates,meanWidth),rnMean(numInfected/disPeriod,meanWidth),'r')
# ax1.plot(rnTime2(curDates,meanWidth),rnMean(SSI_incidens/disPeriod,meanWidth),'g')

ax1.legend(loc='upper left')
ax1.grid(color='k')
# ax1.set_ylim(bottom=0,top=7000)
ax1.set_ylim(bottom=0) 
ax1.set_ylabel('Antal')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

curLastDate = rnTime2(curDates,meanWidth)[-1]
firstDate = np.datetime64('2021-02-01')
ax1.set_xlim([firstDate,curLastDate])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


# Metode 2

In [27]:
dfAG2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
101,2021-05-13,295174,399,364,91.228070,35,8.771930,210,147,189,17237
102,2021-05-14,323429,406,371,91.379310,35,8.620690,219,148,244,18794
103,2021-05-15,263817,346,305,88.150289,41,11.849711,187,115,232,18264
104,2021-05-16,305579,421,383,90.973872,38,9.026128,262,115,276,19956
105,2021-05-17,494948,499,473,94.789579,26,5.210421,283,178,413,27276


In [28]:
AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede'])
AGpospct2Mean = AGpospct2.values[-14:].mean()
AGpospct2 = np.append(AGpospct2,AGpospct2Mean)
AGpospct2 = np.append(AGpospct2,AGpospct2Mean)

In [29]:
AGsens = np.divide(dfAG2['AGpos_PCRpos'], dfAG2['AGpos_PCRpos'] + dfAG2['AGnegPCRpos'])

AGsensmean = AGsens.values[-14:].mean()
AGsens = np.append(AGsens,AGsensmean)
AGsens = np.append(AGsens,AGsensmean)

In [30]:



# AGpospct2
# dfAG.tail()
# AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']).values[-14:].mean()
# AGposProj = np.multiply(dfAG.Tested,AGpospct2)
# AGpospct2Proj = np.divide(AGposProj,dfAG.Tested)


# fig,ax1 = plt.subplots()

# ax1.plot(curDates,AGpospct2Proj)
# ax1.plot(dfAG2['Dato'],np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']))

# len(AGpospct2Proj)

In [31]:
# len(curEstimate)
# curEstimate

In [39]:
fig,ax1 = plt.subplots()


# PCRfirst30000 = dfPCR.NewPositive * ( (30000/dfPCR.Tested)**0.7)
# PCRrest = dfPCR.NewPositive - PCRfirst30000

pcrScreenSimple = dfPCR.Tested - 30000

# avgPosRate = AG_TPR.values[-14:].mean()

# AGpospct2 = (np.divide(dfAG.NewPositive,dfAG.Tested)/avgPosRate).values

# AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede']).values[-14:].mean()
 
# curEstimate = np.multiply(pcrScreenSimple,AGpospct2 * 2)

# curEstimate = np.multiply(pcrScreenSimple.values,AGpospct2 * 2)
# curEstimate = np.multiply(pcrScreenSimple.values,np.divide(AGpospct2,AGsens))

hardCodedSens = 0.618
# hardCodedSens = 0.492
curEstimate = np.multiply(pcrScreenSimple.values,np.divide(AGpospct2,hardCodedSens))


# ax1.plot(curDates,curEstimate,'.-',color='b',linewidth=0.5,markersize=2)
# ax1.plot(rnTime2(curDates,meanWidth),rnMean(curEstimate,meanWidth),'b')

# ax1.plot(curDates,AGposPCRposProj,'.-',color='r',linewidth=0.5,markersize=2)
# ax1.plot(rnTime2(curDates,meanWidth),rnMean(AGposPCRposProj,meanWidth),'r')

# ax1.plot(curDates,AGposPCRposProj+curEstimate,'.-',color='k',linewidth=0.5,markersize=2)
# ax1.plot(rnTime2(curDates,meanWidth),rnMean(AGposPCRposProj+curEstimate,meanWidth),'k')

meanDays = rnTime2(curDates,meanWidth)

# ax1.fill_between(meanDays,rnMean(curEstimate,meanWidth),label='Fundet ved PCR screening')
# ax1.fill_between(meanDays,rnMean(AGposPCRposProj+curEstimate,meanWidth),rnMean(curEstimate,meanWidth),label='Fundet ved AG screening')
# ax1.fill_between(meanDays,rnMean(PCRpos,meanWidth),rnMean(AGposPCRposProj+curEstimate,meanWidth),label='Fundet ved opsporende PCR tests')

ax1.stackplot(meanDays,
    rnMean(PCRpos-AGposPCRposProj - curEstimate,meanWidth),
    rnMean(curEstimate,meanWidth),
    rnMean(AGposPCRposProj,meanWidth),
    labels=['PCR positive, ved 30.000 tests','PCR positive, kun screening','PCR positive, efter AG positive'],
    colors=[cmap(0),cmap(1),cmap(2)]
    )

# ax1.fill_between(meanDays,rnMean(curEstimate,meanWidth),label='Fundet ved PCR screening')
# ax1.fill_between(meanDays,rnMean(AGposPCRposProj+curEstimate,meanWidth),rnMean(curEstimate,meanWidth),label='Fundet ved AG screening')
# ax1.fill_between(meanDays,rnMean(PCRpos,meanWidth),rnMean(AGposPCRposProj+curEstimate,meanWidth),label='Fundet ved opsporende PCR tests')
# # ax1.fill_between(meanDays,rnMean(AGposPCRposProj+curEstimate+PCRnoscreen,meanWidth),rnMean(AGposPCRposProj+curEstimate,meanWidth),label='Fundet ved opsporende PCR tests')

ax1.legend(loc='upper left')
ax1.grid(color='k')
# ax1.set_ylim(bottom=0,top=7000)
ax1.set_ylim(bottom=0) 
ax1.set_ylim(bottom=0,top=1100)
ax1.set_ylabel('Antal')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

curLastDate = rnTime2(curDates,meanWidth)[-1]
firstDate = np.datetime64('2021-03-01')
ax1.set_xlim([firstDate,curLastDate])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Screening_Metode2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
print(f'450 / 3000 = {450 / 3000}')
print(f'800 / 1500 = {800 / 1500}')
print(f'575 / 2250 = {575 / 2250}')

# # 2.3 * 1.5 * 0.5
# 1.7 * 0.8
# (1.7 * 0.8) * (1-0.15)

1.1 * (1 / (1-0.15))
# 1.2 * (1 / (1-0.15))
# 1.3 * (1 / (1-0.15))

450 / 3000 = 0.15
800 / 1500 = 0.5333333333333333
575 / 2250 = 0.25555555555555554


1.2941176470588236

In [28]:

# # PCRnoscreen = PCRpos_nonAG * ( (30000/dfPCR.Tested)**0.7)
# # PCRscreen = PCRpos_nonAG - PCRnoscreen 

# # ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(0),label='PCR positive, ved 30.000 tests')
# # ax1.plot(curDates,PCRscreen,'.-',color='r',linewidth=0.5,markersize=2)
# # ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(1),label='PCR positive, kun screening')

# # ax1.plot(curDates,PCRfirst30000,'.-',color='r',linewidth=0.5,markersize=2) 
# # ax1.plot(curDates,PCRrest,'.-',color='g',linewidth=0.5,markersize=2) 

# ax1.plot(curDates,dfPCR.NewPositive,'.-',color='k',linewidth=0.5,markersize=2) 

# ax1.legend(loc='upper left')
# ax1.grid(color='k')
# # ax1.set_ylim(bottom=0,top=1100)
# ax1.set_ylabel('Antal')

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

# curLastDate = rnTime2(curDates,meanWidth)[-1]
# ax1.set_xlim([firstDate,curLastDate])
# plt.tight_layout()

In [29]:
# firstDate = np.datetime64('2021-03-01')
# lastDate = curDates.iloc[-1]

# fig,ax1 = plt.subplots(1,1,sharex=True)

# meanWidth = 7


# ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust')
# ax1.plot(curDates,AGpos,'.-',color='xkcd:green',linewidth=0.5,markersize=2,label='AG positive, total')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGpos,meanWidth),color='xkcd:green')
# ax1.plot(confDates,AGposPCRpos,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2,label='AG positive, PCR bekræftede')
# ax1.plot(rnTime(confDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:dark green')

# # ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR')
# # ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')
# # ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='PCR positive, due to AG')
# # ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink')


# ax1.legend(loc='upper left')
# # ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax1.grid()
# ax1.set_ylim(bottom=0,top=1400)
# ax1.set_ylabel('Count')

# # ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
# ax1.set_xlim([firstDate,lastDate])
# plt.tight_layout()


# # if saveFigures:
# #     plt.savefig('figs/Viggo_Screening')

# OLD below

In [30]:
fig,ax1 = plt.subplots(1,1)

# ax1.plot(dfAG2.Dato,dfAG2.AG_testede) 
ax1.plot(dfAG2.Dato,np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos),'.-')
ax1.plot(rnTime(dfAG2.Dato,7),rnMean(np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos),7),'--')

truePosRate = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
avgTruePosRate = np.mean(truePosRate[-30:])

ax1.plot(dfAG2.Dato,avgTruePosRate * np.ones(dfAG2.Dato.shape),'k:')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:

fig,ax1 = plt.subplots(1,1)

# PCRpos = dfPCR.NewPositive
# AGpos = dfAG.NewPositive
# ax1.plot(curDates,PCRpos)
ax1.plot(curDates,AGpos,label='AG+')
ax1.plot(curDates,AGpos*avgTruePosRate,label='AG+, times true-positive rate')
ax1.plot(dfAG2.Dato,dfAG2['AGpos_PCRpos'],label='AG+, PCR+ confirmed')

ax1.legend()
ax1.grid()
ax1.set_ylim(bottom=0)
ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:

fig,ax1 = plt.subplots(1,1)

PCRpos_nonAG = PCRpos.values -(AGpos.values*avgTruePosRate)

normFactor = (np.divide(100000,dfPCR['Tested']))**0.7

PCRpos_nonAG_corr = PCRpos_nonAG*normFactor

ax1.plot(curDates,PCRpos,'b.-',linewidth=0.5,markersize=2)
ax1.plot(curDates,PCRpos_nonAG,'m.-',linewidth=0.5,markersize=2)
ax1.plot(curDates,PCRpos_nonAG_corr,'g.-',linewidth=0.5,markersize=2)
meanWidth = 7
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),'b',label='PCR positiv')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'m',label='PCR positiv, uden AG konf.')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG_corr,meanWidth),'g',label='PCR positiv, uden AG konf., korrigeret')
# ax1.plot(curDates,AGpos,label='AG+')
# ax1.plot(curDates,AGpos*avgTruePosRate,label='AG+, times true-positive rate')
# ax1.plot(dfAG2.Dato,dfAG2['AGpos_PCRpos'],label='AG+, PCR+ confirmed')

curFont = 18
ax1.legend(fontsize=curFont)
ax1.grid()
ax1.set_ylim(bottom=0,top=1400)
ax1.set_xlim(left=np.datetime64('2021-03-01'))

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

# plt.xticks(fontsize=curFont,rotation=90)
plt.xticks(fontsize=curFont)
plt.yticks(fontsize=curFont)
ax1.set_ylabel('Antal',fontsize=curFont)
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/PCRKorrigering')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'numpy.ndarray' object has no attribute 'values'